In [1]:
%load_ext autoreload
%autoreload 2
import startup_credit as cr
RAW_CSV      = r"C:\Users\Romain\OneDrive - KU Leuven\Masters\MBIS\Year 2\Semester 2\Statistical Consulting\ExternalData\startup_failures.csv"         # your Crunchbase-style file
OUT_DIR= r"C:\Users\Romain\OneDrive - KU Leuven\Masters\MBIS\Year 2\Semester 2\Statistical Consulting\ExternalData\model" # where to save the output

c:\Users\Romain\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
cr.train(RAW_CSV,OUT_DIR,None)

c:\Users\Romain\OneDrive - KU Leuven\Masters\MBIS\Year 2\Semester 2\Statistical Consulting\startup_credit.py:564: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  out = pd.to_datetime(val, errors="coerce", utc=True)
[I 2025-04-29 18:28:37,978] A new study created in memory with name: no-name-06434082-b0d4-412f-9639-7998b57c4cca
[I 2025-04-29 18:30:13,659] Trial 0 finished with value: 0.7662523852821818 and parameters: {'n_estimators': 600, 'learning_rate': 0.17254716573280354, 'max_depth': 6, 'subsample': 0.8394633936788146, 'colsample_bytree': 0.6624074561769746}. Best is trial 0 with value: 0.7662523852821818.
[I 2025-04-29 18:31:47,898] Trial 1 finished with value: 0.7930137285649845 and parameters: {'n_estimators': 400, 'learning_rate': 0.011900590783184251, 'max_depth': 7, 'subsample': 0.8404460046972835, 'colsample_bytree': 0.8832290311184181}. Best is trial 1 with va

In [ ]:
cr.score(OUT_DIR,RAW_CSV,r"C:\Users\Romain\OneDrive - KU Leuven\Masters\MBIS\Year 2\Semester 2\Statistical Consulting\ExternalData\out.csv")

c:\Users\Romain\OneDrive - KU Leuven\Masters\MBIS\Year 2\Semester 2\Statistical Consulting\startup_credit_5.py:518: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  out = pd.to_datetime(val, errors="coerce", utc=True)
2025-04-29 16:51:07,038 | INFO | PSI vs train: 0.0000
2025-04-29 16:51:07,050 | INFO | K‑S statistic: 0.0000
2025-04-29 16:51:10,976 | INFO | ✔︎ Scored → C:\Users\Romain\OneDrive - KU Leuven\Masters\MBIS\Year 2\Semester 2\Statistical Consulting\ExternalData\out.csv
